<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/SOCOTEC_GPU_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Dataset Creation: Mapping Natural Language Prompts to Function Calls
dataset = [
    {"input": "Calculate the sum of 7 and 3.", "output": "add(7, 3)"},
    {"input": "What's the square of 5?", "output": "square(5)"},
    {"input": "Tell me the cube of 2.", "output": "cube(2)"},
    {"input": "Add 12 to 8.", "output": "add(12, 8)"},
    {"input": "What is 4 squared?", "output": "square(4)"},
    {"input": "Compute the cube of 4.", "output": "cube(4)"},
    {"input": "Say hello to John.", "output": "greet('John')"},
    {"input": "What's 10 + 20?", "output": "add(10, 20)"},
    {"input": "Find the square of 3.", "output": "square(3)"},
    {"input": "Calculate the cube of 5.", "output": "cube(5)"},
    {"input": "Greet Alice for me.", "output": "greet('Alice')"},
    {"input": "What's 2 + 3?", "output": "add(2, 3)"},
    {"input": "Calculate 6 squared.", "output": "square(6)"},
    {"input": "Give me the cube value of 8.", "output": "cube(8)"},
    {"input": "Say hi to Bob.", "output": "greet('Bob')"},
    {"input": "Add 15 and 9.", "output": "add(15, 9)"},
    {"input": "What is 7 cubed?", "output": "cube(7)"},
    {"input": "Calculate 4 cubed.", "output": "cube(4)"},
    {"input": "Square the number 6.", "output": "square(6)"},
    {"input": "Say hello to Sarah.", "output": "greet('Sarah')"},
    {"input": "What is 5 + 5?", "output": "add(5, 5)"},
    {"input": "What is 3 squared?", "output": "square(3)"},
    {"input": "Tell me the cube of 10.", "output": "cube(10)"}
]


In [11]:
# Function Library
def add(a, b):
    """Return the sum of two numbers."""
    return a + b

def square(a):
    """Return the square of a number."""
    return a ** 2

def cube(a):
    """Return the cube of a number."""
    return a ** 3

def greet(name):
    """Return a greeting string."""
    return f"Hello, {name}!"


In [12]:
!pip install transformers accelerate


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Model name as specified
model_name = "Qwen/Qwen2.5-7B-Instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

print("Model and tokenizer successfully loaded!")


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1390: UserWarning: Current model requires 7424 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Model and tokenizer successfully loaded!


In [54]:
# Optimized system prompt tailored for Qwen
system_prompt = """
You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')

You will respond ONLY with the function call in this exact format without any deviation.
Do not add explanations.
"""

def generate_response(prompt, max_tokens=20):
    """
    Generates valid function call responses by simplifying context.
    Ensures prompt is compliant with Qwen 2.5 tokenized expectations.
    """
    # Combine system prompt with user query for context
    full_prompt = f"{system_prompt}\nUser: {prompt}\nResponse:"

    # Tokenize the entire prompt string
    input_tokens = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=512)

    # Generate output based on tokenized input
    output_tokens = model.generate(
      **input_tokens,
      max_new_tokens=max_tokens,
      do_sample=False,  # Correctly disable sampling for deterministic behavior
      temperature=0.7,  # Use a small temperature value like 0.7 if needed for variation
      pad_token_id=tokenizer.pad_token_id
  )


    # Decode model's response
    response = tokenizer.decode(output_tokens[0], skip_special_tokens=True).strip()

    # Debug response
    print(f"Decoded response: '{response}'")

    # Validate response strictly
    if response.startswith(("add", "square", "cube", "greet")):
        return response
    else:
        return "Error: Invalid response."

In [55]:
# Test the model
response = generate_response("What's the square of 5?")
print("Response from model:", response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Decoded response: 'You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')

You will respond ONLY with the function call in this exact format without any deviation.
Do not add explanations.

User: What's the square of 5?
Response: square(5)'
Response from model: Error: Invalid response.


In [56]:
def add(a, b):
    return a + b

def square(a):
    return a ** 2

def cube(a):
    return a ** 3

def greet(name):
    return f"Hello, {name}!"


In [57]:
import re

def parse_and_execute(response):
    """
    Parses the response from the LLM and executes the corresponding function safely.
    :param response: The response string from the LLM.
    :return: Result of the parsed function execution.
    """
    try:
        if match := re.search(r"add\((\d+),\s*(\d+)\)", response):
            a, b = int(match.group(1)), int(match.group(2))
            return add(a, b)

        if match := re.search(r"square\((\d+)\)", response):
            num = int(match.group(1))
            return square(num)

        if match := re.search(r"cube\((\d+)\)", response):
            num = int(match.group(1))
            return cube(num)

        if match := re.search(r"greet\(['\"]([^'\"]+)['\"]\)", response):
            name = match.group(1)
            return greet(name)

        return "Function not recognized or invalid input"

    except Exception as e:
        return f"Error processing response: {e}"


In [58]:
# Complete interaction pipeline
user_input = "What's the square of 4?"

# Generate response from model
response = generate_response(user_input)

# Parse and execute the response
result = parse_and_execute(response)

print("Response from model:", response)
print("Executed result:", result)


Decoded response: 'You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')

You will respond ONLY with the function call in this exact format without any deviation.
Do not add explanations.

User: What's the square of 4?
Response: square(4)'
Response from model: Error: Invalid response.
Executed result: Function not recognized or invalid input


In [59]:
test_prompts = [
    "Calculate the sum of 7 and 3.",
    "Tell me the cube of 2.",
    "What's the square of 5?",
    "Greet Alice",
]

for prompt in test_prompts:
    response = generate_response(prompt)
    result = parse_and_execute(response)
    print(f"Prompt: {prompt}")
    print(f"Model Response: {response}")
    print(f"Execution Result: {result}")
    print("-" * 50)


Decoded response: 'You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')

You will respond ONLY with the function call in this exact format without any deviation.
Do not add explanations.

User: Calculate the sum of 7 and 3.
Response: add(7, 3)'
Prompt: Calculate the sum of 7 and 3.
Model Response: Error: Invalid response.
Execution Result: Function not recognized or invalid input
--------------------------------------------------
Decoded response: 'You are a specialized assistant that only generates valid function calls as defined below:
Functions allowe

In [60]:
# Optimized system prompt tailored for Qwen
system_prompt = """
You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')

You will respond ONLY with the function call in this exact format without any deviation.
Do not add explanations.
"""

# LLM Response Generator
def generate_response(prompt, max_tokens=30):
    """
    Generates model responses, validates tokenized output.
    Handles edge case issues gracefully.
    """
    try:
        # Combine system context with user input
        full_prompt = f"{system_prompt}\nUser: {prompt}\nResponse:"

        # Tokenize and prepare input
        input_tokens = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=512)

        # Generate the response
        output_tokens = model.generate(
            **input_tokens,
            max_new_tokens=max_tokens,
            do_sample=False,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
        )

        # Decode response safely
        response = tokenizer.decode(output_tokens[0], skip_special_tokens=True).strip()
        print(f"Decoded response: '{response}'")

        # Check if response adheres to expected patterns
        if response.startswith(("add", "square", "cube", "greet")):
            return response
        else:
            return "Error: Invalid response."
    except Exception as e:
        return f"Model Error: {e}"


# Function to safely parse and execute model responses
import re

def parse_and_execute(response):
    """
    Parses LLM response and executes the corresponding function safely.
    """
    try:
        # Regex matching logic for parsing responses
        if match := re.search(r"add\((\d+),\s*(\d+)\)", response):
            a, b = int(match.group(1)), int(match.group(2))
            return add(a, b)

        if match := re.search(r"square\((\d+)\)", response):
            num = int(match.group(1))
            return square(num)

        if match := re.search(r"cube\((\d+)\)", response):
            num = int(match.group(1))
            return cube(num)

        if match := re.search(r"greet\(['\"]([^'\"]+)['\"]\)", response):
            name = match.group(1)
            return greet(name)

        # Default invalid response
        return "Function not recognized or invalid input"
    except Exception as e:
        return f"Error parsing response: {e}"


# Test pipeline with model prompts
user_prompts = [
    "What's the square of 4?",
    "Tell me the cube of 8.",
    "Calculate 7 + 3.",
    "Say hi to Bob."
]

for prompt in user_prompts:
    print("-" * 40)
    print(f"User Input: {prompt}")
    model_response = generate_response(prompt)
    execution_result = parse_and_execute(model_response)
    print(f"Model Response: {model_response}")
    print(f"Execution Result: {execution_result}")

----------------------------------------
User Input: What's the square of 4?
Decoded response: 'You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')

You will respond ONLY with the function call in this exact format without any deviation.
Do not add explanations.

User: What's the square of 4?
Response: square(4)'
Model Response: Error: Invalid response.
Execution Result: Function not recognized or invalid input
----------------------------------------
User Input: Tell me the cube of 8.
Decoded response: 'You are a specialized assistant that only generat

In [61]:
# System prompt tailored specifically to limit outputs strictly
system_prompt = """
You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')
"""

# Function to handle LLM response safely
def generate_response(prompt, max_tokens=30):
    """
    Generates a response limited strictly to the user's query
    using the system prompt and avoids context bleed.
    """
    try:
        # Combine the system prompt and user query
        full_prompt = f"{system_prompt}\nUser: {prompt}\nResponse:"

        # Tokenize the prompt and pass it to the model
        input_tokens = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=512)

        # Generate a clean response with a limited token range
        output_tokens = model.generate(
            **input_tokens,
            max_new_tokens=max_tokens,
            do_sample=False,  # deterministic output
            temperature=0.3,  # Low temperature for deterministic responses
            pad_token_id=tokenizer.pad_token_id,
        )

        # Decode only the generated response tokens
        decoded_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True).strip()

        # Debugging: Ensure the decoded response is visible
        print("Decoded Response: ", decoded_response)

        # Validate response strictly
        if decoded_response.startswith(("add(", "square(", "cube(", "greet(")):
            return decoded_response
        else:
            return "Error: Invalid response format"
    except Exception as e:
        return f"Model or system error: {e}"


# Regex handler for parsing results
def parse_and_execute(response):
    """
    Executes regex-safe response matching and validates the LLM output.
    Handles common functions like add, square, cube, greet safely.
    """
    try:
        # Add regex safety layers
        if match := re.search(r"add\((\d+),\s*(\d+)\)", response):
            a, b = int(match.group(1)), int(match.group(2))
            return add(a, b)

        if match := re.search(r"square\((\d+)\)", response):
            num = int(match.group(1))
            return square(num)

        if match := re.search(r"cube\((\d+)\)", response):
            num = int(match.group(1))
            return cube(num)

        if match := re.search(r"greet\(['\"]([^'\"]+)['\"]\)", response):
            name = match.group(1)
            return greet(name)

        # Catch-all for unexpected function calls
        return "Unknown or malformed response."
    except Exception as e:
        return f"Error parsing response: {e}"


# Mock function examples
def add(a, b):
    return a + b

def square(a):
    return a ** 2

def cube(a):
    return a ** 3

def greet(name):
    return f"Hello, {name}!"


# Test pipeline
user_inputs = [
    "What's the square of 4?",
    "Tell me the cube of 8.",
    "Calculate 7 + 3.",
    "Say hi to Bob."
]

for input_text in user_inputs:
    print("-" * 40)
    print(f"User Input: {input_text}")
    model_response = generate_response(input_text)  # Get the model response
    execution_result = parse_and_execute(model_response)  # Handle regex and execute safely
    print(f"Model Response: {model_response}")
    print(f"Execution Result: {execution_result}")

----------------------------------------
User Input: What's the square of 4?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Decoded Response:  You are a specialized assistant that only generates valid function calls as defined below:
Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

You MUST only output function calls without explanations, context, or extra text. Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Greet Alice."
Response: greet('Alice')

User: What's the square of 4?
Response: square(4) add(7, 3)
Model Response: Error: Invalid response format
Execution Result: Unknown or malformed response.
----------------------------------------
User Input: Tell me the cube of 8.


KeyboardInterrupt: 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

# Initialize the Model and Tokenizer
model_name = "unsloth/Llama-3.2-11B-Vision-Instruct"  # Using the Llama-3.3-70B-Instruct model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")  # Use device map for large models

# Optimized system prompt with tuned instruction clarity
system_prompt = """
You are a highly specialized, rule-based assistant that generates ONLY valid function calls as defined below. Do NOT add context, explanations, or additional text unless specified.

Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Say hi to Alice."
Response: greet('Alice')

Instructions:
- Always match the response exactly to these allowed function formats.
- Do not deviate from the examples provided.
- Ensure no extraneous context or text is added.
"""

# Function to handle LLM response safely
def generate_response(prompt, max_tokens=30):
    """
    Generates a response limited strictly to the user's query
    using the system prompt and avoids context bleed.
    """
    try:
        # Combine the system prompt and user query
        full_prompt = f"{system_prompt}\nUser: {prompt}\nResponse:"

        # Tokenize the prompt and pass it to the model
        input_tokens = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=512)

        # Generate a clean response with a limited token range
        output_tokens = model.generate(
            **input_tokens,
            max_new_tokens=max_tokens,
            do_sample=False,  # deterministic output
            temperature=0.3,  # Low temperature for deterministic responses
            pad_token_id=tokenizer.pad_token_id,
        )

        # Decode only the generated response tokens
        decoded_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True).strip()

        # Debugging: Ensure the decoded response is visible
        print("Decoded Response: ", decoded_response)

        # Validate response strictly
        if decoded_response.startswith(("add(", "square(", "cube(", "greet(")):
            return decoded_response
        else:
            return "Error: Invalid response format"
    except Exception as e:
        return f"Model or system error: {e}"


# Regex handler for parsing results
def parse_and_execute(response):
    """
    Executes regex-safe response matching and validates the LLM output.
    Handles common functions like add, square, cube, greet safely.
    """
    try:
        # Add regex safety layers
        if match := re.search(r"add\((\d+),\s*(\d+)\)", response):
            a, b = int(match.group(1)), int(match.group(2))
            return add(a, b)

        if match := re.search(r"square\((\d+)\)", response):
            num = int(match.group(1))
            return square(num)

        if match := re.search(r"cube\((\d+)\)", response):
            num = int(match.group(1))
            return cube(num)

        if match := re.search(r"greet\(['\"]([^'\"]+)['\"]\)", response):
            name = match.group(1)
            return greet(name)

        # Catch-all for unexpected function calls
        return "Unknown or malformed response."
    except Exception as e:
        return f"Error parsing response: {e}"


# Mock function examples
def add(a, b):
    return a + b

def square(a):
    return a ** 2

def cube(a):
    return a ** 3

def greet(name):
    return f"Hello, {name}!"


# Test pipeline
user_inputs = [
    "What's the square of 4?",
    "Tell me the cube of 8.",
    "Calculate 7 + 3.",
    "Say hi to Bob."
]

for input_text in user_inputs:
    print("-" * 40)
    print(f"User Input: {input_text}")
    model_response = generate_response(input_text)  # Get the model response
    execution_result = parse_and_execute(model_response)  # Handle regex and execute safely
    print(f"Model Response: {model_response}")
    print(f"Execution Result: {execution_result}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of MllamaForCausalLM were not initialized from the model checkpoint at unsloth/Llama-3.2-11B-Vision-Instruct and are newly initialized: ['embed_tokens.weight', 'layers.0.input_layernorm.weight', 'layers.0.mlp.down_proj.weight', 'layers.0.mlp.gate_proj.weight', 'layers.0.mlp.up_proj.weight', 'layers.0.post_attention_layernorm.weight', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.o_proj.weight', 'layers.0.self_attn.q_proj.weight', 'layers.0.self_attn.v_proj.weight', 'layers.1.input_layernorm.weight', 'layers.1.mlp.down_proj.weight', 'layers.1.mlp.gate_proj.weight', 'layers.1.mlp.up_proj.weight', 'layers.1.post_attention_layernorm.weight', 'layers.1.self_attn.k_proj.weight', 'layers.1.self_attn.o_proj.weight', 'layers.1.self_attn.q_proj.weight', 'layers.1.self_attn.v_proj.weight', 'layers.10.input_layernorm.weight', 'layers.10.mlp.down_proj.weight', 'layers.10.mlp.gate_proj.weight', 'layers.10.mlp.up_proj.weight', 'layers.10.post_attention_layernorm.weight', 'layer

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling f

----------------------------------------
User Input: What's the square of 4?
Decoded Response:  You are a highly specialized, rule-based assistant that generates ONLY valid function calls as defined below. Do NOT add context, explanations, or additional text unless specified.

Functions allowed:
- add(a, b): Adds two integers
- square(a): Squares a given integer
- cube(a): Cubes a given integer
- greet(name): Returns a greeting for a given name

Examples:
User: "What's 5 squared?"
Response: square(5)

User: "Calculate 7 + 3."
Response: add(7,3)

User: "Say hi to Alice."
Response: greet('Alice')

Instructions:
- Always match the response exactly to these allowed function formats.
- Do not deviate from the examples provided.
- Ensure no extraneous context or text is added.

User: What's the square of 4?
Response:!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Model Response: Error: Invalid response format
Execution Result: Unknown or malformed response.
----------------------------------------
User Input